## Modelo LSTM

In [2]:
"""
Script: train_model.py
Descrição:
  - Coleta dados históricos da Petrobras (PETR3.SA) usando yfinance
  - Prepara dados para previsão multi-step (prever 10 dias à frente)
  - Cria e treina um modelo LSTM com saída de 10 neurônios
  - Salva o modelo e o scaler para uso na inferência
"""

# Importações necessárias
import yfinance as yf
import numpy as np
import pandas as pd
import datetime
import pickle

# Bibliotecas de Machine Learning / Deep Learning
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Dense
from tensorflow.keras.callbacks import EarlyStopping


In [3]:
# Parâmetros
TICKER = "PETR3.SA"
START_DATE = "2010-01-01"
END_DATE = datetime.date.today().strftime("%Y-%m-%d")
TIME_STEPS = 60         # Número de dias passados usados como entrada
FORECAST_HORIZON = 10   # Número de dias futuros que queremos prever

# 1. Coleta de dados
print("Baixando dados do Yahoo Finance...")
df = yf.download(TICKER, start=START_DATE, end=END_DATE)

# Vamos usar apenas a coluna 'Close'
df = df[['Close']].dropna()  # remove possíveis NaN

# 2. Normalização dos dados
scaler = MinMaxScaler(feature_range=(0, 1))
data_scaled = scaler.fit_transform(df.values)  # shape: (num_samples, 1)

# 3. Criação das janelas de 60 dias para prever 10 dias
X, y = [], []

# Queremos X[i] = 60 valores passados, y[i] = 10 valores seguintes
# Precisamos garantir que haja 10 valores após a janela
# Logo, o loop vai até len(data_scaled) - TIME_STEPS - FORECAST_HORIZON + 1
for i in range(TIME_STEPS, len(data_scaled) - FORECAST_HORIZON + 1):
    # A sequência de entrada: do (i - 60) até (i-1)
    X.append(data_scaled[i - TIME_STEPS : i, 0])
    # A sequência de saída: do (i) até (i + 10 - 1)
    y.append(data_scaled[i : i + FORECAST_HORIZON, 0])

# Converte para NumPy
X = np.array(X)  # shape: (samples, 60)
y = np.array(y)  # shape: (samples, 10)

# Redimensiona X para (samples, 60, 1) (LSTM espera 3D)
X = X.reshape((X.shape[0], X.shape[1], 1))
# y fica como (samples, 10)

# 4. Divisão em treino e teste
split_index = int(0.8 * len(X))
X_train, X_test = X[:split_index], X[split_index:]
y_train, y_test = y[:split_index], y[split_index:]

print(f"Tamanho do conjunto de treinamento: {X_train.shape}")
print(f"Tamanho do conjunto de teste: {X_test.shape}")

Baixando dados do Yahoo Finance...


[*********************100%%**********************]  1 of 1 completed

Tamanho do conjunto de treinamento: (2949, 60, 1)
Tamanho do conjunto de teste: (738, 60, 1)


In [4]:
df.tail()

,Close
Date,
2025-02-10,40.029999
2025-02-11,40.250000
2025-02-12,39.320000
2025-02-13,39.430000
2025-02-14,40.849998


In [23]:
# 5. Construção do modelo LSTM
model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(TIME_STEPS, 1)))
model.add(Dropout(0.2))
model.add(LSTM(50, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(50))
model.add(Dropout(0.2))
# Camada de saída com 10 neurônios para prever 10 dias à frente
model.add(Dense(FORECAST_HORIZON))

model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae', 'mape', 'mse', 'accuracy'])

In [24]:
# Callback para early stopping
early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# 6. Treinamento
print("Treinando o modelo LSTM para previsão de 10 dias à frente...")
history = model.fit(
    X_train, y_train,
    epochs=100,
    batch_size=32,
    validation_data=(X_test, y_test),
    callbacks=[early_stop]
)

Treinando o modelo LSTM para previsão de 10 dias à frente...
Epoch 1/100
93/93 ━━━━━━━━━━━━━━━━━━━━ 8s 57ms/step - accuracy: 0.0861 - loss: 0.0531 - mae: 0.1673 - mape: 27522.6543 - mse: 0.0531 - val_accuracy: 0.1707 - val_loss: 0.0070 - val_mae: 0.0683 - val_mape: 8.5857 - val_mse: 0.0070
Epoch 2/100
93/93 ━━━━━━━━━━━━━━━━━━━━ 5s 49ms/step - accuracy: 0.0994 - loss: 0.0095 - mae: 0.0726 - mape: 14549.8848 - mse: 0.0095 - val_accuracy: 0.0718 - val_loss: 0.0117 - val_mae: 0.0928 - val_mape: 11.5189 - val_mse: 0.0117
Epoch 3/100
93/93 ━━━━━━━━━━━━━━━━━━━━ 5s 49ms/step - accuracy: 0.0908 - loss: 0.0075 - mae: 0.0639 - mape: 9710.8232 - mse: 0.0075 - val_accuracy: 0.1762 - val_loss: 0.0065 - val_mae: 0.0661 - val_mape: 8.2959 - val_mse: 0.0065
Epoch 4/100
93/93 ━━━━━━━━━━━━━━━━━━━━ 5s 52ms/step - accuracy: 0.0941 - loss: 0.0064 - mae: 0.0585 - mape: 14614.5674 - mse: 0.0064 - val_accuracy: 0.1829 - val_loss: 0.0047 - val_mae: 0.0548 - val_mape: 7.0198 - val_mse: 0.0047
Epoch 5/100
93/93 ━

In [26]:

# 7. Avaliação rápida (opcional)
print("Avaliando o modelo no conjunto de teste...")
loss, mae, mape, mse = model.evaluate(X_train, y_train)
print(f"Loss (MSE) no treinamento: {loss}")
print(f"MAE no treinamento: {mae}")
print(f"MAPE no treinamento: {mape}")
print(f"MSE no treinamento: {mse}")


print('-'*100)
loss, mae, mape, mse = model.evaluate(X_test, y_test)
print(f"Loss (MSE) no teste: {loss}")
print(f"MAE no teste: {mae}")
print(f"MAPE no teste: {mape}")
print(f"MSE no teste: {mse}")

Avaliando o modelo no conjunto de teste...
93/93 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - accuracy: 0.1861 - loss: 0.0013 - mae: 0.0278 - mape: 6320.9209 - mse: 0.0013


ValueError: too many values to unpack (expected 4)

### Tentando ajustar o modelo com keras

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping

# Exemplo de ajuste de hiperparâmetros:
# - Aumentamos o número de neurônios para 64
# - Utilizamos dropout e recurrent_dropout diretamente na camada LSTM
# - Inserimos camadas de BatchNormalization para estabilizar os gradientes
# - Aumentamos o número de épocas para dar mais tempo de treinamento

model = Sequential()
model.add(LSTM(64, return_sequences=True, 
               input_shape=(TIME_STEPS, 1),
               dropout=0.2, recurrent_dropout=0.2))
model.add(BatchNormalization())

model.add(LSTM(64, return_sequences=True, 
               dropout=0.2, recurrent_dropout=0.2))
model.add(BatchNormalization())

model.add(LSTM(64, dropout=0.2, recurrent_dropout=0.2))
model.add(BatchNormalization())

# Camada de saída para prever FORECAST_HORIZON (por exemplo, 10 dias à frente)
model.add(Dense(FORECAST_HORIZON))

# Compilação do modelo: removemos 'accuracy' e mantemos métricas relevantes para regressão
model.compile(optimizer='adam', 
              loss='mean_squared_error', 
              metrics=['mae', 'mape', 'mse'])

# Callback para early stopping, monitorando a perda de validação
early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

print("Treinando o modelo LSTM otimizado para previsão de 10 dias à frente...")
history = model.fit(
    X_train, y_train,
    epochs=100,  # Aumentamos o número de épocas para melhor convergência
    batch_size=32,
    validation_data=(X_test, y_test),
    callbacks=[early_stop]
)


Treinando o modelo LSTM otimizado para previsão de 10 dias à frente...
Epoch 1/10
93/93 ━━━━━━━━━━━━━━━━━━━━ 15s 93ms/step - loss: 0.7407 - mae: 0.6441 - mape: 272471.4375 - mse: 0.7407 - val_loss: 0.3040 - val_mae: 0.5306 - val_mape: 67.4635 - val_mse: 0.3040
Epoch 2/10
93/93 ━━━━━━━━━━━━━━━━━━━━ 8s 85ms/step - loss: 0.0626 - mae: 0.1965 - mape: 91728.6953 - mse: 0.0626 - val_loss: 0.2016 - val_mae: 0.4311 - val_mape: 54.3798 - val_mse: 0.2016
Epoch 3/10
93/93 ━━━━━━━━━━━━━━━━━━━━ 9s 92ms/step - loss: 0.0303 - mae: 0.1355 - mape: 106489.4844 - mse: 0.0303 - val_loss: 0.1306 - val_mae: 0.3445 - val_mape: 43.1797 - val_mse: 0.1306
Epoch 4/10
93/93 ━━━━━━━━━━━━━━━━━━━━ 8s 88ms/step - loss: 0.0243 - mae: 0.1184 - mape: 87452.9922 - mse: 0.0243 - val_loss: 0.1399 - val_mae: 0.3597 - val_mape: 45.4625 - val_mse: 0.1399
Epoch 5/10
93/93 ━━━━━━━━━━━━━━━━━━━━ 8s 86ms/step - loss: 0.0224 - mae: 0.1140 - mape: 134544.7031 - mse: 0.0224 - val_loss: 0.0977 - val_mae: 0.2988 - val_mape: 37.6343 - v

In [20]:
# 7. Avaliação rápida (opcional)
print("Avaliando o modelo no conjunto de teste...")
loss, mae, mape, mse  = model.evaluate(X_train, y_train)
print(f"Loss (MSE) no treinamento: {loss}")
print(f"MAE no treinamento: {mae}")
print(f"MAPE no treinamento: {mape}")
print(f"MSE no treinamento: {mse}")


print('-'*100)
loss, mae, mape, mse  = model.evaluate(X_test, y_test)
print(f"Loss (MSE) no teste: {loss}")
print(f"MAE no teste: {mae}")
print(f"MAPE no teste: {mape}")
print(f"MSE no teste: {mse}")


Avaliando o modelo no conjunto de teste...
93/93 ━━━━━━━━━━━━━━━━━━━━ 2s 26ms/step - loss: 0.0118 - mae: 0.0939 - mape: 76844.2109 - mse: 0.0118
Loss (MSE) no treinamento: 0.01265661045908928
MAE no treinamento: 0.09568314254283905
MAPE no treinamento: 106913.640625
MSE no treinamento: 0.01265661045908928
----------------------------------------------------------------------------------------------------
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - loss: 0.0239 - mae: 0.1395 - mape: 18.5129 - mse: 0.0239
Loss (MSE) no teste: 0.03304833918809891
MAE no teste: 0.16654856503009796
MAPE no teste: 20.674577713012695
MSE no teste: 0.03304833918809891


### Anda otimizando com keras

In [21]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

model = Sequential()

# Primeira camada LSTM com 128 neurônios, dropout e recurrent_dropout de 0.3
model.add(LSTM(128, return_sequences=True, input_shape=(TIME_STEPS, 1), 
               dropout=0.3, recurrent_dropout=0.3))
model.add(BatchNormalization())

# Segunda camada LSTM com 128 neurônios
model.add(LSTM(128, return_sequences=True, 
               dropout=0.3, recurrent_dropout=0.3))
model.add(BatchNormalization())

# Terceira camada LSTM com 128 neurônios (sem return_sequences)
model.add(LSTM(128, dropout=0.3, recurrent_dropout=0.3))
model.add(BatchNormalization())

# Camada densa intermediária para refinar o aprendizado
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.3))

# Camada de saída para prever FORECAST_HORIZON (ex: 10 dias)
model.add(Dense(FORECAST_HORIZON))

# Compilação do modelo - removemos "accuracy", pois não é adequada para regressão
model.compile(optimizer='adam', 
              loss='mean_squared_error', 
              metrics=['mae', 'mape', 'mse'])

# Callbacks: EarlyStopping e redução dinâmica da taxa de aprendizado
early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
lr_reduce = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6)

print("Treinando o modelo LSTM otimizado para previsão de 10 dias à frente...")
history = model.fit(
    X_train, y_train,
    epochs=100,           # Aumentamos as épocas para permitir melhor convergência
    batch_size=32,
    validation_data=(X_test, y_test),
    callbacks=[early_stop, lr_reduce]
)


Treinando o modelo LSTM otimizado para previsão de 10 dias à frente...
Epoch 1/100
93/93 ━━━━━━━━━━━━━━━━━━━━ 21s 151ms/step - loss: 0.6763 - mae: 0.6149 - mape: 414389.2812 - mse: 0.6763 - val_loss: 0.3874 - val_mae: 0.6092 - val_mape: 77.9845 - val_mse: 0.3874 - learning_rate: 0.0010
Epoch 2/100
93/93 ━━━━━━━━━━━━━━━━━━━━ 16s 174ms/step - loss: 0.0703 - mae: 0.2093 - mape: 116704.3359 - mse: 0.0703 - val_loss: 0.2987 - val_mae: 0.5340 - val_mape: 68.2374 - val_mse: 0.2987 - learning_rate: 0.0010
Epoch 3/100
93/93 ━━━━━━━━━━━━━━━━━━━━ 16s 174ms/step - loss: 0.0424 - mae: 0.1612 - mape: 96240.3516 - mse: 0.0424 - val_loss: 0.2120 - val_mae: 0.4462 - val_mape: 56.6551 - val_mse: 0.2120 - learning_rate: 0.0010
Epoch 4/100
93/93 ━━━━━━━━━━━━━━━━━━━━ 16s 170ms/step - loss: 0.0402 - mae: 0.1590 - mape: 112326.1328 - mse: 0.0402 - val_loss: 0.2221 - val_mae: 0.4606 - val_mape: 58.7739 - val_mse: 0.2221 - learning_rate: 0.0010
Epoch 5/100
93/93 ━━━━━━━━━━━━━━━━━━━━ 18s 191ms/step - loss: 0.03

In [22]:
# 7. Avaliação rápida (opcional)
print("Avaliando o modelo no conjunto de teste...")
loss, mae, mape, mse  = model.evaluate(X_train, y_train)
print(f"Loss (MSE) no treinamento: {loss}")
print(f"MAE no treinamento: {mae}")
print(f"MAPE no treinamento: {mape}")
print(f"MSE no treinamento: {mse}")


print('-'*100)
loss, mae, mape, mse  = model.evaluate(X_test, y_test)
print(f"Loss (MSE) no teste: {loss}")
print(f"MAE no teste: {mae}")
print(f"MAPE no teste: {mape}")
print(f"MSE no teste: {mse}")

Avaliando o modelo no conjunto de teste...
93/93 ━━━━━━━━━━━━━━━━━━━━ 4s 38ms/step - loss: 0.0100 - mae: 0.0795 - mape: 83180.0000 - mse: 0.0100
Loss (MSE) no treinamento: 0.010876620188355446
MAE no treinamento: 0.08257120847702026
MAPE no treinamento: 115686.140625
MSE no treinamento: 0.010876620188355446
----------------------------------------------------------------------------------------------------
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 37ms/step - loss: 0.0220 - mae: 0.1321 - mape: 17.5151 - mse: 0.0220
Loss (MSE) no teste: 0.028304418548941612
MAE no teste: 0.15256135165691376
MAPE no teste: 18.971134185791016
MSE no teste: 0.028304418548941612


### Tentando a otimização com o torch

In [28]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import DataLoader, TensorDataset

# Definição de hiperparâmetros
TIME_STEPS = 30          # Por exemplo, 30 passos de tempo
FORECAST_HORIZON = 10    # Previsão de 10 dias à frente
INPUT_DIM = 1            # Número de features (por exemplo, apenas o preço)
HIDDEN_SIZE = 128
DENSE_UNITS = 64
BATCH_SIZE = 32
EPOCHS = 100
LEARNING_RATE = 0.001
EARLY_STOP_PATIENCE = 10

# Dispositivo (GPU se disponível)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Modelo LSTM com camadas intermediárias e Batch Normalization
class LSTMForecastModel(nn.Module):
    def __init__(self, time_steps, forecast_horizon, input_dim=1, hidden_size=128, dense_units=64):
        super(LSTMForecastModel, self).__init__()
        # Primeira camada LSTM
        self.lstm1 = nn.LSTM(input_dim, hidden_size, num_layers=1, batch_first=True)
        self.bn1 = nn.BatchNorm1d(hidden_size)
        
        # Segunda camada LSTM
        self.lstm2 = nn.LSTM(hidden_size, hidden_size, num_layers=1, batch_first=True)
        self.bn2 = nn.BatchNorm1d(hidden_size)
        
        # Terceira camada LSTM
        self.lstm3 = nn.LSTM(hidden_size, hidden_size, num_layers=1, batch_first=True)
        self.bn3 = nn.BatchNorm1d(hidden_size)
        
        # Camada densa intermediária
        self.fc1 = nn.Linear(hidden_size, dense_units)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.3)
        
        # Camada de saída para previsão do horizonte desejado
        self.fc2 = nn.Linear(dense_units, forecast_horizon)
        
    def forward(self, x):
        # x: (batch, TIME_STEPS, INPUT_DIM)
        out, _ = self.lstm1(x)
        # Aplicando BatchNorm: precisamos transpor para (batch, features, seq_len)
        out = out.transpose(1, 2)
        out = self.bn1(out)
        out = out.transpose(1, 2)
        out = self.dropout(out)
        
        out, _ = self.lstm2(out)
        out = out.transpose(1, 2)
        out = self.bn2(out)
        out = out.transpose(1, 2)
        out = self.dropout(out)
        
        out, _ = self.lstm3(out)
        out = out.transpose(1, 2)
        out = self.bn3(out)
        out = out.transpose(1, 2)
        out = self.dropout(out)
        
        # Seleciona a saída do último timestep
        out = out[:, -1, :]  # (batch, hidden_size)
        out = self.fc1(out)
        out = self.relu(out)
        out = self.dropout(out)
        out = self.fc2(out)   # (batch, forecast_horizon)
        return out

# Instancia o modelo e o envia para o dispositivo
model = LSTMForecastModel(TIME_STEPS, FORECAST_HORIZON, INPUT_DIM, HIDDEN_SIZE, DENSE_UNITS)
model.to(device)

# Definição da função de perda e do otimizador
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5, min_lr=1e-6)

# Supondo que você já possua os dados de treinamento e validação como tensores:
# X_train, y_train, X_val, y_val
# Eles devem ter as formas:
# X_train: (num_amostras_train, TIME_STEPS, INPUT_DIM)
# y_train: (num_amostras_train, FORECAST_HORIZON)
# (o mesmo para os dados de validação)

import torch
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
X_val = torch.tensor(X_val, dtype=torch.float32)
y_val = torch.tensor(y_val, dtype=torch.float32)

# Cria DataLoaders para treinamento e validação
train_dataset = TensorDataset(X_train, y_train)
val_dataset = TensorDataset(X_val, y_val)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

# Loop de treinamento com early stopping
best_val_loss = float('inf')
patience_counter = 0

for epoch in range(EPOCHS):
    model.train()
    train_loss = 0.0
    for batch_X, batch_y in train_loader:
        batch_X = batch_X.to(device)
        batch_y = batch_y.to(device)
        
        optimizer.zero_grad()
        predictions = model(batch_X)
        loss = criterion(predictions, batch_y)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item() * batch_X.size(0)
    train_loss /= len(train_loader.dataset)
    
    # Validação
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for batch_X, batch_y in val_loader:
            batch_X = batch_X.to(device)
            batch_y = batch_y.to(device)
            predictions = model(batch_X)
            loss = criterion(predictions, batch_y)
            val_loss += loss.item() * batch_X.size(0)
    val_loss /= len(val_loader.dataset)
    
    # Ajusta a taxa de aprendizado com base na perda de validação
    scheduler.step(val_loss)
    
    print(f"Época {epoch+1}/{EPOCHS} - Treino: {train_loss:.4f} - Validação: {val_loss:.4f}")
    
    # Early Stopping
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        patience_counter = 0
        best_model_state = model.state_dict()
    else:
        patience_counter += 1
        if patience_counter >= EARLY_STOP_PATIENCE:
            print("Early stopping acionado.")
            break

# Carrega o melhor modelo obtido
model.load_state_dict(best_model_state)

NameError: name 'X_val' is not defined

In [ ]:
print('-'*100)
# 8. Salvando o modelo e o scaler
print("Salvando o modelo em 'lstm_model_petr3.h5'...")
model.save("lstm_model_petr3.h5")

print("Salvando o scaler em 'scaler.pkl'...")
with open("scaler.pkl", "wb") as f:
    pickle.dump(scaler, f)

print("Treinamento concluído e arquivos salvos com sucesso.")                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           